# GENERATE DATASET

In [ ]:
#1. Generate dataset
#2. Train the classifier and save it
#3. Detect the face and named it if it is already stored in our dataset

In [1]:
import cv2
import os

In [2]:
def generate_dataset():
    face_classifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    face_cascade_profile = cv2.CascadeClassifier("haarcascade_profileface.xml")
    def face_cropped(img):
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray,1.4,5)
        #scaling factor = 1.3
        #Minimum neighbor = 5

        if len(faces) == 0:
            faces = face_cascade_profile.detectMultiScale(gray, 1.4, 5)
            if len(faces) == 0:
                flipped_gray = cv2.flip(gray, 1)
                faces = face_cascade_profile.detectMultiScale(gray, 1.4, 5)

        if len(faces) == 0:
            return None
        for (x,y,w,h) in faces:
            cropped_face = img[y:y+h, x:x+w]
        return cropped_face

    cap = cv2.VideoCapture(0)
    id = 1
    img_id = 0 

    while True:
        ret, frame = cap.read()
        if face_cropped(frame) is not None:
            img_id += 1
            face = cv2.resize(face_cropped(frame), (300,300))
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
            file_name_path = "data/user." + str(id) + "." + str(img_id) + ".jpg"
            cv2.imwrite(file_name_path, face)
            cv2.putText(face, str(img_id), (50,50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
            # (50,50) is the origin point from where text is to be written
            # font scale = 1
            # thickness = 2
            cv2.imshow("Cropped face", face)
            if cv2.waitKey(1) == 15 or int(img_id) == 100:
                break

    cap.release()
    cv2.destroyAllWindows()
    print("Collecting samples is completed ....")

generate_dataset()
            

2025-04-25 17:58:05.641 Python[84970:16666516] +[IMKClient subclass]: chose IMKClient_Modern
2025-04-25 17:58:05.641 Python[84970:16666516] +[IMKInputSession subclass]: chose IMKInputSession_Modern


In [3]:
%pip install pillow
%pip install numpy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
import cv2
from PIL import Image #pip install pillow
import numpy as np    # pip install numpy
 
def train_classifier(data_dir):
    path = [os.path.join(data_dir, f) for f in os.listdir(data_dir)]
     
    faces = []
    ids = []
     
    for image in path:
        img = Image.open(image).convert('L')
        imageNp = np.array(img, 'uint8')
        id = int(os.path.split(image)[1].split(".")[1])
         
        faces.append(imageNp)
        ids.append(id)
         
    ids = np.array(ids)
     
    # Train and save classifier
    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.train(faces,ids)
    clf.write("classifier.xml")
train_classifier("data")

In [ ]:
import cv2
import numpy as np
from PIL import Image
import os
 
def draw_boundary(img, classifier, profile_classifier, scaleFactor, minNeighbors, color, text, clf):
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    features = classifier.detectMultiScale(gray_img, scaleFactor, minNeighbors)

    if len(features) == 0:
        features = profile_classifier.detectMultiScale(gray_img, scaleFactor, minNeighbors)
        if len(features) == 0:
            flipped_gray = cv2.flip(gray_img, 1)
            features = profile_classifier.detectMultiScale(flipped_gray, scaleFactor, minNeighbors)
     
    for (x,y,w,h) in features:
        cv2.rectangle(img, (x,y), (x+w,y+h), color, 2 )
         
        id, pred = clf.predict(gray_img[y:y+h,x:x+w])
        confidence = int(100*(1-pred/300))
         
        if confidence>70:
            if id==1:
                cv2.putText(img, "Xuan Phuc", (x,y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
            if id==2:
                cv2.putText(img, "Cristiano Ronaldo", (x,y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
        else:
            cv2.putText(img, "UNKNOWN", (x,y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,255), 1, cv2.LINE_AA)
     
    return img
 
# loading classifier
faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
profileCascade = cv2.CascadeClassifier("haarcascade_profileface.xml")

clf = cv2.face.LBPHFaceRecognizer_create()
clf.read("classifier.xml")
 
video_capture = cv2.VideoCapture(0)
 
while True:
    ret, img = video_capture.read()
    img = draw_boundary(img, faceCascade, profileCascade, 1.4, 6, (0, 255, 0), "Face", clf)
    cv2.imshow("face Detection", img)
     
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
video_capture.release()
cv2.destroyAllWindows()

error: OpenCV(4.11.0) /Users/xperience/GHA-Actions-OpenCV/_work/opencv-python/opencv-python/opencv/modules/imgproc/src/color.cpp:199: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


: 